In [1]:
import gym

In [2]:
env = gym.make('SpaceInvaders-v0')

In [3]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
    print('Episode: {}\nScore: {}'.format(episode, score))

env.close()
    

Episode: 1
Score: 35.0
Episode: 2
Score: 125.0
Episode: 3
Score: 185.0
Episode: 4
Score: 315.0
Episode: 5
Score: 180.0
Episode: 6
Score: 110.0
Episode: 7
Score: 155.0
Episode: 8
Score: 110.0
Episode: 9
Score: 30.0


In [4]:
#import Neural network packages
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [11]:
del model

In [12]:
model = build_model(height, width, channels, actions)

In [13]:
#importing keras-rL2 reinforcement learning functions
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy  import LinearAnnealedPolicy, EpsGreedyQPolicy

In [14]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [15]:
dqn = build_agent(model, actions)

In [16]:
dqn.compile(Adam(lr=0.0001))

C:\Users\murra\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [17]:
dqn.fit(env, nb_steps=40000, visualize=False, verbose=1)

Training for 40000 steps ...
Interval 1 (0 steps performed)


C:\Users\murra\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 155625s 16s/step - reward: 0.1950
15 episodes - episode_reward: 130.000 [35.000, 385.000] - loss: 1.477 - mean_q: -0.301 - mean_eps: 0.505 - ale.lives: 2.040

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 18019s 2s/step - reward: 0.2245
13 episodes - episode_reward: 163.846 [50.000, 490.000] - loss: 0.557 - mean_q: 0.039 - mean_eps: 0.100 - ale.lives: 2.072

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 412426s 41s/step - reward: 0.2640
12 episodes - episode_reward: 229.583 [40.000, 545.000] - loss: 1.157 - mean_q: 0.956 - mean_eps: 0.100 - ale.lives: 2.037

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 10036s 1s/step - reward: 0.2240
done, took 596108.235 seconds


In [20]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 320.000, steps: 1066
Episode 2: reward: 45.000, steps: 538
Episode 3: reward: 155.000, steps: 545
Episode 4: reward: 200.000, steps: 793
Episode 5: reward: 105.000, steps: 500
Episode 6: reward: 205.000, steps: 687
Episode 7: reward: 95.000, steps: 757
Episode 8: reward: 35.000, steps: 372
Episode 9: reward: 105.000, steps: 540
Episode 10: reward: 125.000, steps: 695
139.0
